In [13]:
from models.adapters.base import BaseAdapter
from models.selector import FilterSelector
from models.quantization import conv2d_outter, conv2d_core
from experiments.planner import CGPPinPlanner
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import pandas as pd
import numpy as np

# Disable LaTeX rendering
plt.rc('text', usetex=False)

data_store = Path("../../data_store/lenet")
data_store.mkdir(exist_ok=True, parents=True)
loaded = False
sns.set_theme(context="paper", style="whitegrid", rc={"text.usetex" : True})
sns.set_palette("deep")
sns.color_palette("deep")

[(0.2980392156862745, 0.4470588235294118, 0.6901960784313725),
 (0.8666666666666667, 0.5176470588235295, 0.3215686274509804),
 (0.3333333333333333, 0.6588235294117647, 0.40784313725490196),
 (0.7686274509803922, 0.3058823529411765, 0.3215686274509804),
 (0.5058823529411764, 0.4470588235294118, 0.7019607843137254),
 (0.5764705882352941, 0.47058823529411764, 0.3764705882352941),
 (0.8549019607843137, 0.5450980392156862, 0.7647058823529411),
 (0.5490196078431373, 0.5490196078431373, 0.5490196078431373),
 (0.8, 0.7254901960784313, 0.4549019607843137),
 (0.39215686274509803, 0.7098039215686275, 0.803921568627451)]

In [4]:
adapter = BaseAdapter.load_base_model("qat_quantized_lenet", "./trained_models/qat.quantized_lenet.pth")

c:\Users\Majo\.conda\envs\masters\Lib\site-packages\torch\ao\quantization\observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
c:\Users\Majo\.conda\envs\masters\Lib\site-packages\torch\ao\quantization\observer.py:1207: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(
c:\Users\Majo\.conda\envs\masters\Lib\site-packages\torch\_utils.py:376: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


In [23]:
assert isinstance(adapter, BaseAdapter)
data = []
# while not data or data[-1][2][1] >= 0.75:
for e in range(-128, 127, 1):
    adapter = BaseAdapter.load_base_model("qat_quantized_lenet", "./trained_models/qat.quantized_lenet.pth")
    # layers = [adapter.get_layer(layer) for layer in ["conv1"]]
    for layer in ["conv1", "conv2"]:
        weights, bias = adapter.get_train_weights(layer), adapter.get_bias(layer)
        planner = CGPPinPlanner()
        
        for filter_i, filter_weights in enumerate(weights):
            for channel_i, channel_filters in enumerate(filter_weights):
                for selector in conv2d_core([filter_i, channel_i], 5, 3):
                    weights[*selector] = weights[*selector] + e
        
        adapter.inject_weights([weights.flatten()], [[FilterSelector(layer, [], [[slice(None), slice(None), slice(None), slice(None)]])]], inline=True)
    top_k, avg_loss = adapter.evaluate(top=[1, 5])
    data.append([e] + list(top_k.values()) + [avg_loss])
    
loaded = True
    

c:\Users\Majo\.conda\envs\masters\Lib\site-packages\torch\ao\quantization\observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
c:\Users\Majo\.conda\envs\masters\Lib\site-packages\torch\ao\quantization\observer.py:1207: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(
Loss: 11.3653, Acc: 0.101500, Top-5: 0.433100: 100%|██████████| 1/1 [00:02<00:00,  2.29s/batch]


In [2]:
if not loaded:
    dfs = []
    for file in ["lenet.256.csv", "lenet.outter.256.csv", "lenet.inner.256.csv"]:
        x = pd.read_csv(data_store / file)
        x["Type"] = "Outter" if "outter" in file else "Inner" if "inner" in file else "All"
        dfs.append(x)
        
    df = pd.concat(dfs)
    df.columns = [col.title() for col in df.columns]
    df["Type"] = df["Type"].str.title()
else:
    df = pd.DataFrame(data=data, columns=["error", "top-1", "top-5", "loss"])
    df.to_csv(data_store / "lenet.inner.256.csv", index=False)

In [16]:
def get_x(y_value: int, point_a, point_b):
    x1, y1 = point_a
    x2, y2 = point_b
    x_hat = None
    if y1 <= y_value <= y2 and y2 <= y_value <= y1:
        m = (y2-y1) / (x2-x1)
        b = y1 - m * x1
        x_hat = (y_value - b) / m
    
    return x_hat
        

def plot_line(g, value, label):
    x_min = float("inf")
    x_max = float("-inf")
    ax = g.axes
    # for ax in g.axes.flat:
    #     ax = ax
    for line in g.axes.lines:
        # for line in ax.lines:
        x, y = line.get_data()
        if isinstance(x, list) or x.size == 0 or y.size == 0: continue
        
        for a, b in zip(zip(x[0:-2], y[0:-2]), zip(x[1:], y[1:])):
            x_hat = get_x(value, a, b)
            
            if x_hat is None:
                continue
            
            x_min = min([x_min, x_hat])
            x_max = max([x_max, x_hat])                    
            
    ax.text(x_min, value, f" {x_min:.1f} ({label})", ha="left", va="top", alpha=0.6)
    ax.text(x_max, value, f" {x_max:.1f} ({label})", ha="left", va="top", alpha=0.6)
    ax.axhline(value, linestyle="--", color="black", linewidth=1, alpha=0.75)    
    # ax.axvline(x_min, linestyle="--", color="green", linewidth=1, alpha=0.75)    
    # ax.axvline(x_max, linestyle="--", color="green", linewidth=1, alpha=0.75)
    ax.scatter(x_min, value, color="black", alpha=0.75, marker="x", linewidth=1)
    ax.scatter(x_max, value, color="black", alpha=0.75, marker="x", linewidth=1)
    # ax.(x_max, linestyle="--", color="green", linewidth=1, alpha=0.75)    
        # ax.axhline(value, linestyle="--", color="#cfcfcf", lw=2, alpha=0.75)    

filtered_df = df.loc[df["Top-1"] >= 0, :]
value_vars = ["Top-1"]
y = "Accuracy"
target = filtered_df.loc[filtered_df["Error"] == 0, "Top-1"].max()

plot_df = pd.melt(filtered_df, id_vars=["Error", "Type"], value_vars=value_vars, var_name="Metric", value_name="Accuracy")

fig, (ax1) = plt.subplots()
g1 = sns.lineplot(data=plot_df, x="Error", y=y, hue="Type", markers=True, ax=ax1)
g1.set(xlabel="Error per Weight")
plt.legend(title="Error by Type")
# g1.xaxis.set_major_locator(MultipleLocator(5))
plot_line(g1, target * 0.95, "α = 0.05")
plot_line(g1, target * 0.90, "α = 0.10")

sns.despine(fig, bottom=True, left=True)

# ax1.get_legend().remove()

# plot_line(ax, 0.90286, "MobileNetV2 top-5")
# plt.xlim(plot_df["error"].min(), plot_df["error"].max())
plt.show
plt.savefig(data_store / "lenet.256.error.svg")
plt.savefig(data_store / "lenet.256.error.png")
plt.savefig(data_store / "lenet.256.error.pdf")



c:\Users\Majo\.conda\envs\masters\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
c:\Users\Majo\.conda\envs\masters\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpw9gv3_2a c9cfdea0164fb35b195621b9469c7849.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./c9cfdea0164fb35b195621b9469c7849.tex
LaTeX2e <2023-06-01> patch level 1
L3 programming layer <2023-08-29>
(c:/Users/Majo/AppData/Roaming/TinyTeX/texmf-dist/tex/latex/base/article.cls
Document Class: article 2023/05/17 v1.4n Standard LaTeX document class
(c:/Users/Majo/AppData/Roaming/TinyTeX/texmf-dist/tex/latex/base/size10.clo))

! LaTeX Error: File `type1cm.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.8 \usepackage
               {type1ec}^^M
No pages of output.
Transcri

Error in callback <function _draw_all_if_interactive at 0x0000025920555440> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpz3spay_c c9cfdea0164fb35b195621b9469c7849.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./c9cfdea0164fb35b195621b9469c7849.tex
LaTeX2e <2023-06-01> patch level 1
L3 programming layer <2023-08-29>
(c:/Users/Majo/AppData/Roaming/TinyTeX/texmf-dist/tex/latex/base/article.cls
Document Class: article 2023/05/17 v1.4n Standard LaTeX document class
(c:/Users/Majo/AppData/Roaming/TinyTeX/texmf-dist/tex/latex/base/size10.clo))

! LaTeX Error: File `type1cm.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.8 \usepackage
               {type1ec}^^M
No pages of output.
Transcri

RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpjnzg5ya6 c9cfdea0164fb35b195621b9469c7849.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./c9cfdea0164fb35b195621b9469c7849.tex
LaTeX2e <2023-06-01> patch level 1
L3 programming layer <2023-08-29>
(c:/Users/Majo/AppData/Roaming/TinyTeX/texmf-dist/tex/latex/base/article.cls
Document Class: article 2023/05/17 v1.4n Standard LaTeX document class
(c:/Users/Majo/AppData/Roaming/TinyTeX/texmf-dist/tex/latex/base/size10.clo))

! LaTeX Error: File `type1cm.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.8 \usepackage
               {type1ec}^^M
No pages of output.
Transcri

<Figure size 640x480 with 1 Axes>

In [5]:
a_df = filtered_df.loc[(filtered_df["Type"] == "Outter") & (filtered_df["Top-1"] >= 0.95), :]
b_df = filtered_df.loc[(filtered_df["Type"] == "Outter") & (filtered_df["Top-1"] >= 0.9) & (filtered_df["Top-1"] < 0.95), :]
c_df = filtered_df.loc[(filtered_df["Type"] == "Outter") & (filtered_df["Top-1"] >= 0.8) & (filtered_df["Top-1"] < 0.9), :]
d_df = filtered_df.loc[(filtered_df["Type"] == "Outter") & (filtered_df["Top-1"] >= 0.7) & (filtered_df["Top-1"] < 0.8), :]
d_df = filtered_df.loc[(filtered_df["Type"] == "Outter") & (filtered_df["Top-1"] >= 0.25) & (filtered_df["Top-1"] < 0.7), :]

a_df["Error backup"] = a_df["Error"]
b_df["Error backup"] = b_df["Error"]
c_df["Error backup"] = c_df["Error"]
d_df["Error backup"] = c_df["Error"]

a_min_idx = a_df["Error"].apply(lambda x: abs(x)).idxmax()
b_min_idx = b_df["Error"].apply(lambda x: abs(x)).idxmax()
c_min_idx = c_df["Error"].apply(lambda x: abs(x)).idxmax()
d_min_idx = d_df["Error"].apply(lambda x: abs(x)).idxmax()

a_min_row = a_df.loc[a_min_idx]
b_min_row = b_df.loc[b_min_idx]
c_min_row = c_df.loc[c_min_idx]
d_min_row = d_df.loc[d_min_idx]

print(a_min_row, b_min_row, c_min_row, d_min_row)


Error                -11
Top-1             0.9534
Top-5             0.9988
Loss            0.147825
Type              Outter
Error backup         -11
Name: 117, dtype: object Error                -14
Top-1             0.9006
Top-5             0.9926
Loss            0.341971
Type              Outter
Error backup         -14
Name: 114, dtype: object Error                -16
Top-1             0.8369
Top-5             0.9842
Loss            0.568979
Type              Outter
Error backup         -16
Name: 112, dtype: object Error                 31
Top-1             0.2525
Top-5             0.8591
Loss            15.82792
Type              Outter
Error backup         NaN
Name: 159, dtype: object


C:\Users\Majo\AppData\Local\Temp\ipykernel_4404\1387617683.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a_df["Error backup"] = a_df["Error"]
C:\Users\Majo\AppData\Local\Temp\ipykernel_4404\1387617683.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b_df["Error backup"] = b_df["Error"]
C:\Users\Majo\AppData\Local\Temp\ipykernel_4404\1387617683.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se